In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import os, sys
from time import perf_counter

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

export_DIR = "../Shot_Charts/ShotLocationData/"

In [ ]:
# Only 1 season
year = 2023
league, league_id, season = "NBA", "00", str(year) + "-" + str(year + 1)[-2:]
player_dict = get_players(league=league, from_year=year, to_year=year)
player_ids = [player["pID"] for player in player_dict]
len(player_ids)

In [ ]:
shota = []
for player_id in tqdm(player_ids):
    for ii in range(5):
        # time.sleep(0.6)
        try:
            player_shotchart = ShotChartDetail(
                league_id=league_id,
                team_id=0,
                player_id=player_id,
                context_measure_simple="FGA",
                season_nullable=season,
            )
            shots_df = player_shotchart.get_data_frames()[0]
            shots_df["LOC_X"] = -shots_df["LOC_X"]
            shota.append(shots_df)
            break
        except Exception as error:
            print(player_id)
            print(error)
            time.sleep(0.6)
            continue
shots = pd.concat(shota)
# shots.to_csv(export_DIR+ league + f"_Shot_Details_{year}.csv", index=False)
shots.to_parquet(export_DIR+ league + f"_Shot_Details_{year}.parquet")

In [ ]:
league = "NBA"
for year in range(2000,2024):
    # shots = pd.read_csv( export_DIR + league + f"_Shot_Details_{year}.csv")
    shots = pd.read_parquet(export_DIR + league + f"_Shot_Details_{year}.parquet")